In [5]:
import numpy as np
import tensorflow as tf

class Audiobooks_Data_Reader():
    
    # Initialization method. If batch size is not given, it is default to None
    def __init__(self, dataset, batch_size = None):
        # Load the dataset. npz files have been named coherently so one line of code works for all files.
        npz = np.load('Audiobooks_data_{0}.npz'.format(dataset))
        
        # Two variables which take the values of the inputs and targets. Inputs here are floats and targets are ints.
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        # Checks the batch number. If batch number is None. then we are either validating or testing,
        # which is done in a single batch.
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    # Method which loads the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
        
        # Slicing the dataset into batches and letting the enxt function load them one after another
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        # One hot econde targets. If more than 2 classes, change the number of the variable.
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        # Function returns the input batch and the one hot encoded targets
        return inputs_batch, targets_one_hot
    
    def __iter__(self):
        return self

### Create the machine learning algorithm

In [15]:
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50

# Resets any variables left in memory from previous runs.
tf.reset_default_graph()

# Declare placeholders where the variable will be put in
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.int32, [None, output_size])

# Weights and biases for the first linear combination between inputs and first hiddne layer.
# get_variable makes use of the default TensorFlow initializer which is Xavier initialization.
weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])
# Operation between inputs and first hidden layer. ReLu was used as activation function.
outputs_1 = tf.nn.tanh(tf.matmul(inputs,weights_1) + biases_1)

# Weights and biases for the first linear combination between first hidden layer and second hidden layer.
weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
# Operation between first hidden layer and second hidden layer. ReLu was used again here.
outputs_2 = tf.nn.tanh(tf.matmul(outputs_1, weights_2) + biases_2)

# Weights and biases for the final combination.
weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])
# Final operation between second hidden layer and outputs. ReLu wasn't used here
# because we use a trick to include it directly into the loss function. Works for softmax and sigmoid.
outputs = tf.matmul(outputs_2,weights_3) + biases_3

# Calculates the loss function for every output or pair.
loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs, labels = targets)

# Gets the average loss.
mean_loss = tf.reduce_mean(loss)

# Optimization stage defined below.
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

# Get a 0 or 1 for every input in the batch.
out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

# Get the average accuracy of the outputs.
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

# Declare session variable.
session = tf.InteractiveSession()

# Initialize the variables. Default is Xavier.
initializer = tf.global_variables_initializer()

# Run the session.
session.run(initializer)

# Batch size
batch_size = 100


# Basic early stopping.
max_epochs = 50

# Keep track of the validation loss of the previous epoch.
# The variable is kept at very high number to make sure we dont trigger stop
# in the first epoch.
previous_validation_loss = 9999999.

train_data = Audiobooks_Data_Reader('train', batch_size)
validation_data = Audiobooks_Data_Reader('validation')

for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.
    
    for input_batch, target_batch in train_data:
        _, batch_loss = session.run([optimize, mean_loss],
             feed_dict={inputs: input_batch, targets: target_batch})
        
        curr_epoch_loss += batch_loss
    
    curr_epoch_loss /= train_data.batch_count
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = session.run([mean_loss, accuracy],
             feed_dict={inputs: input_batch, targets: target_batch})
        
    print('Epoch ' + str(epoch_counter + 1) +
         '. Training loss: ' + '{0:.3f}'.format(curr_epoch_loss) +
         '. Validation loss: ' + '{0:.3f}'.format(validation_loss) +
         '. Validation accuracy: ' + '{0:.2f}'.format(validation_accuracy * 100) + '%')
    
    if validation_loss > previous_validation_loss:
        break
    previous_validation_loss = validation_loss

print('End of training.')

D:\AnacondaIDE\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 1. Training loss: 0.562. Validation loss: 0.444. Validation accuracy: 75.39%
Epoch 2. Training loss: 0.426. Validation loss: 0.393. Validation accuracy: 78.52%
Epoch 3. Training loss: 0.398. Validation loss: 0.375. Validation accuracy: 78.08%
Epoch 4. Training loss: 0.385. Validation loss: 0.363. Validation accuracy: 78.30%
Epoch 5. Training loss: 0.376. Validation loss: 0.355. Validation accuracy: 79.64%
Epoch 6. Training loss: 0.369. Validation loss: 0.348. Validation accuracy: 79.64%
Epoch 7. Training loss: 0.364. Validation loss: 0.342. Validation accuracy: 80.31%
Epoch 8. Training loss: 0.359. Validation loss: 0.337. Validation accuracy: 81.21%
Epoch 9. Training loss: 0.356. Validation loss: 0.333. Validation accuracy: 81.21%
Epoch 10. Training loss: 0.352. Validation loss: 0.329. Validation accuracy: 81.21%
Epoch 11. Training loss: 0.349. Validation loss: 0.326. Validation accuracy: 81.66%
Epoch 12. Training loss: 0.346. Validation loss: 0.323. Validation accuracy: 81.66%
E

### Test the model

In [16]:
test_data = Audiobooks_Data_Reader('test')
for input_batch, target_batch in (test_data):
    test_accuracy = session.run([accuracy],
                           feed_dict={inputs: input_batch, targets: target_batch})

test_accuracy_percent = test_accuracy[0] * 100.

print('Test accuracy: ' + '{0:.2f}'.format(test_accuracy_percent) + '%')

Test accuracy: 84.60%
